In [ ]:
import os
from tqdm import tqdm, trange
import time
import re
import copy
import gc

import numpy as np
import pandas as pd

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn as nn
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn import functional as F

from transformers import BartForConditionalGeneration,AutoTokenizer, BartConfig
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention

from dataset import KoBARTSummaryDataset

In [3]:
data_path='/content/drive/MyDrive/Dacon/한국어 생성요약/data/235813_AI 기반 회의 녹취록 요약 경진대회_data/'
model_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_model/'
sub_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_sub/'

# LongformerKoBART

In [ ]:
# replaces Kobart's attention layer

class LongformerSelfAttentionForBart(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.embed_dim = config.d_model
        self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
        self.output = nn.Linear(self.embed_dim, self.embed_dim)


    # be able to receive the same type of input as the existing layer of kobart and make the same type of output
    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        # bs x seq_len x seq_len -> bs x seq_len 으로 변경
        attention_mask = attention_mask.squeeze(dim=1)
        attention_mask = attention_mask[:,0]

        is_index_masked = attention_mask < 0
        is_index_global_attn = attention_mask > 0
        is_global_attn = is_index_global_attn.flatten().any().item()

        outputs = self.longformer_self_attn(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=None,
            is_index_masked=is_index_masked,
            is_index_global_attn=is_index_global_attn,
            is_global_attn=is_global_attn,
            output_attentions=output_attentions,
        )

        attn_output = self.output(outputs[0])

        return (attn_output,) + outputs[1:] if len(outputs) == 2 else (attn_output, None, None)

In [ ]:
# code used when calling from_pretrained after saving the model
# redefine the embedding layer because it is not created according to the config

class LongformerEncoderDecoderForConditionalGeneration(BartForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        
        if config.attention_mode == 'n2':
            pass  # do nothing, use BertSelfAttention instead
        else:

            self.model.encoder.embed_positions = BartLearnedPositionalEmbedding(
                config.max_encoder_position_embeddings, 
                config.d_model, 
                config.pad_token_id)

            self.model.decoder.embed_positions = BartLearnedPositionalEmbedding(
                config.max_decoder_position_embeddings, 
                config.d_model, 
                config.pad_token_id)

            for i, layer in enumerate(self.model.encoder.layers):
                layer.self_attn = LongformerSelfAttentionForBart(config, layer_id=i)

In [ ]:
# longformer bart model config creation class

class LongformerEncoderDecoderConfig(BartConfig):
    def __init__(self, attention_window: List[int] = None, attention_dilation: List[int] = None,
                 autoregressive: bool = False, attention_mode: str = 'sliding_chunks',
                 gradient_checkpointing: bool = False, **kwargs):
        """
        Args:
            attention_window: list of attention window sizes of length = number of layers.
                window size = number of attention locations on each side.
                For an affective window size of 512, use `attention_window=[256]*num_layers`
                which is 256 on each side.
            attention_dilation: list of attention dilation of length = number of layers.
                attention dilation of `1` means no dilation.
            autoregressive: do autoregressive attention or have attention of both sides
            attention_mode: 'n2' for regular n^2 self-attention, 'tvm' for TVM implemenation of Longformer
                selfattention, 'sliding_chunks' for another implementation of Longformer selfattention
        """
        super().__init__(**kwargs)
        self.attention_window = attention_window
        self.attention_dilation = attention_dilation
        self.autoregressive = autoregressive
        self.attention_mode = attention_mode
        self.gradient_checkpointing = gradient_checkpointing
        assert self.attention_mode in ['tvm', 'sliding_chunks', 'n2']

In [ ]:
args={}
args['train_path']=data_path+'train_evi_concat_final.csv'
args['test_path']=data_path+'test_evi_final.csv'
args['weight_path']=model_path
args['batch_size']=4
args['num_workers']=4
args['max_epochs']=4
args['attention_window'] = 512
args['max_pos'] = 2052
args['max_seq_len'] = 2048

In [ ]:
# load tokenizer, model
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart", model_max_length=args['max_pos'])
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")
config = LongformerEncoderDecoderConfig.from_pretrained("hyunwoongko/kobart")

model.config = config

# in BART attention_probs_dropout_prob is attention_dropout, but LongformerSelfAttention
# expects attention_probs_dropout_prob, so set it here
config.attention_probs_dropout_prob = config.attention_dropout
config.architectures = ['LongformerEncoderDecoderForConditionalGeneration']

# extend position embeddings
tokenizer.model_max_length = args['max_pos']
tokenizer.init_kwargs['model_max_length'] = args['max_pos']
current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
assert current_max_pos == config.max_position_embeddings + 2

config.max_encoder_position_embeddings = args['max_pos']
config.max_decoder_position_embeddings = config.max_position_embeddings
del config.max_position_embeddings
args['max_pos'] += 2  # NOTE: BART has positions 0,1 reserved, so embedding size is max position + 2
assert args['max_pos'] >= current_max_pos

# allocate a larger position embedding matrix for the encoder
new_encoder_pos_embed = model.model.encoder.embed_positions.weight.new_empty(args['max_pos'], embed_size)
# copy position embeddings over and over to initialize the new position embeddings
k = 2
step = current_max_pos - 2
while k < args['max_pos'] - 1:
    new_encoder_pos_embed[k:(k + step)] = model.model.encoder.embed_positions.weight[2:]
    k += step
model.model.encoder.embed_positions.weight.data = new_encoder_pos_embed

# allocate a larger position embedding matrix for the decoder
# new_decoder_pos_embed = model.model.decoder.embed_positions.weight.new_empty(args['max_pos'], embed_size)
# # copy position embeddings over and over to initialize the new position embeddings
# k = 2
# step = current_max_pos - 2
# while k < args['max_pos'] - 1:
#     new_decoder_pos_embed[k:(k + step)] = model.model.decoder.embed_positions.weight[2:]
#     k += step
# model.model.decoder.embed_positions.weight.data = new_decoder_pos_embed

# replace the `modeling_bart.SelfAttention` object with `LongformerSelfAttention`
config.attention_window = [args['attention_window']] * config.num_hidden_layers
config.attention_dilation = [1] * config.num_hidden_layers

for i, layer in enumerate(model.model.encoder.layers):
    longformer_self_attn_for_bart = LongformerSelfAttentionForBart(config, layer_id=i)

    longformer_self_attn_for_bart.longformer_self_attn.query = layer.self_attn.q_proj
    longformer_self_attn_for_bart.longformer_self_attn.key = layer.self_attn.k_proj
    longformer_self_attn_for_bart.longformer_self_attn.value = layer.self_attn.v_proj

    longformer_self_attn_for_bart.longformer_self_attn.query_global = copy.deepcopy(layer.self_attn.q_proj)
    longformer_self_attn_for_bart.longformer_self_attn.key_global = copy.deepcopy(layer.self_attn.k_proj)
    longformer_self_attn_for_bart.longformer_self_attn.value_global = copy.deepcopy(layer.self_attn.v_proj)

    longformer_self_attn_for_bart.output = layer.self_attn.out_proj

    layer.self_attn = longformer_self_attn_for_bart

In [ ]:
torch.manual_seed(1514)

train_data = KoBARTSummaryDataset(args['train_path'],tok=AutoTokenizer.from_pretrained("hyunwoongko/kobart"),max_len=args['max_seq_len'])
train_data, eval_data = torch.utils.data.random_split(train_data,[2400,594])

train_dataloader = DataLoader(train_data,
                              batch_size=args['batch_size'], 
                              shuffle=True,
                              num_workers=args['num_workers'], 
                              pin_memory=True)

eval_dataloader = DataLoader(eval_data,
                              batch_size=args['batch_size'], 
                              shuffle=False,
                              num_workers=args['num_workers'], 
                              pin_memory=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
def accuracy_function(real, pred):
    accuracies = torch.eq(real, torch.argmax(pred, dim=2))
    mask = torch.logical_not(torch.eq(real, -100))
    accuracies = torch.logical_and(mask, accuracies)
    accuracies = accuracies.clone().detach()
    mask = mask.clone().detach()

    return torch.sum(accuracies)/torch.sum(mask)

def loss_function(real, pred):
    mask = torch.logical_not(torch.eq(real, -100))
    loss_ = criterion(pred.permute(0,2,1), real)
    mask = mask.clone().detach()
    loss_ = mask * loss_

    return torch.sum(loss_)/torch.sum(mask)

In [ ]:
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),lr=2e-5, weight_decay=1e-4,correct_bias=False)
scaler = amp.GradScaler()


for epoch_i in range(0, args['max_epochs']):
  
  # ========================================
  #               Training
  # ========================================

  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, args['max_epochs']))
  model.train()
  t0 = time.time()
  total_train_loss = 0
  total_train_acc = 0
  total_batch=len(train_dataloader)

  for i, batch in enumerate(train_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    decoder_input_ids = batch['decoder_input_ids'].to(device)
    decoder_attention_mask = batch['decoder_attention_mask'].to(device)
    labels = batch['labels'].to(device)

    model.model.encoder.config.gradient_checkpointing = True
    model.model.decoder.config.gradient_checkpointing = True
    
    optimizer.zero_grad()

    with amp.autocast():
      output = model(input_ids=input_ids,
                   attention_mask=attention_mask,
                   decoder_input_ids=decoder_input_ids,
                   decoder_attention_mask=decoder_attention_mask,
                   labels=labels)
      loss = output.loss

    acc = accuracy_function(labels, output.logits)

    total_train_acc += acc  
    total_train_loss += loss

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
      
    training_time = time.time() - t0

    print(f"\rTotal Batch {i+1}/{total_batch} , elapsed time : {training_time/(i+1):.1f}s , train_loss : {total_train_loss/(i+1):.2f}, train_acc: {total_train_acc/(i+1):.3f}", end='')
  print("")

  model.eval()
  total_eval_loss=0
  total_val_acc=0
  total_val_batch=len(eval_dataloader)

  for i,batch in enumerate(eval_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    decoder_input_ids = batch['decoder_input_ids'].to(device)
    decoder_attention_mask = batch['decoder_attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
      output = model(input_ids=input_ids,
                   attention_mask=attention_mask,
                   decoder_input_ids=decoder_input_ids,
                   decoder_attention_mask=decoder_attention_mask,
                   labels=labels)
      loss = output.loss
    
    acc = accuracy_function(labels, output.logits)
    
    total_val_acc +=acc
    total_eval_loss += loss
    
    print(f"\rValidation Batch {i+1}/{total_val_batch} , validation loss: {total_eval_loss/(i+1):.2f}, val_acc: {total_val_acc/(i+1):.3f}", end='')

  torch.save(model.state_dict(), args['weight_path']+'LongformerKoBART_2048_epoch {} weight.ckpt'.format(epoch_i + 1))


======== Epoch 1 / 4 ========
Total Batch 600/600 , elapsed time : 1.2s , train_loss : 0.98, train_acc: 0.810
Validation Batch 149/149 , validation loss: 0.83, val_acc: 0.832
======== Epoch 2 / 4 ========
Total Batch 600/600 , elapsed time : 1.2s , train_loss : 0.56, train_acc: 0.873
Validation Batch 149/149 , validation loss: 0.82, val_acc: 0.836
======== Epoch 3 / 4 ========
Total Batch 600/600 , elapsed time : 1.2s , train_loss : 0.40, train_acc: 0.900
Validation Batch 149/149 , validation loss: 0.85, val_acc: 0.836
======== Epoch 4 / 4 ========
Total Batch 600/600 , elapsed time : 1.2s , train_loss : 0.30, train_acc: 0.922
Validation Batch 149/149 , validation loss: 0.88, val_acc: 0.836

# Generation

+ Generate(sets, max_length=200, no_repeat_ngram_size=3)

In [ ]:
model.load_state_dict(torch.load(args['weight_path']+'LongformerKoBART_2048_epoch 2 weight.ckpt'))
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): LongformerSelfAttentionForBart(
            (longformer_self_attn): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
      

In [ ]:
# test data load
test_data = KoBARTSummaryDataset(args['test_path'],tok=AutoTokenizer.from_pretrained("hyunwoongko/kobart"),max_len=args['max_len'],infer=True)

test_dataloader = DataLoader(test_data,
                              batch_size=args['batch_size'], 
                              shuffle=False,
                              num_workers=args['num_workers'], 
                              pin_memory=True)

In [ ]:
# prediction
test_sum=[]

for i,batch in tqdm(enumerate(test_dataloader)):
  sets=batch['input_ids'].to(device)
  batch_sum=model.generate(sets, max_length=200,no_repeat_ngram_size=3)
  test_sum=[*test_sum,*batch_sum]

127it [01:21,  1.57it/s]


In [ ]:
test_sum_sent=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in test_sum]

sub=pd.read_csv(data_path+'sample_submission.csv')
sub['summary']=test_sum_sent

In [ ]:
sub.to_csv(sub_path+'LongformerKoBART_2048_epoch 2.csv',index=False)

# View sub

In [4]:
test_real=pd.read_csv(data_path+'test_evi_final.csv')
sub=pd.read_csv(sub_path+'kobart_epoch2_ml_1024_evi_v02_g3.csv')

In [7]:
id=27
print('Context:\n')
display(test_real['title_context'][id])

print('\nsummary:\n')
display(sub['summary'][id])

Context:



'제296회 본회의 제1차: 의사일정 제9항, 음성군 참전유공자 지원 조례 일부개정조례안, 의사일정10항, 음성군 국가보훈대상자 예우 및 지원에 관한 조례 일부개정조례안을 일괄 상정합니다. 주민지원과장께서는 나오셔서 2건의 안건에 대하여 일괄 제안설명하여 주시기 바랍니다. 주민지원과장입니다. 음성군 참전유공자 지원 조례 일부개정조례안에 대하여 설명드리겠습니다. 개정이유는 참전유공자와 그 유족에게 합당한 예우를 하기 위하여 월남참전유공자에게 참전명예수당을 매월 10만원으로 인상하여 지급하고 참전유공자가 사망한 경우 그 배우자에게 매월 5만원을 지급하는 내용입니다. 주요 내용으로는 월남참전유공자 참전명예수당의 경우 2018년 2월 9일 조례ㆍ규칙심의회에서 수정 가결되어 매월 10만원으로 2018년도 1월부터 소급하여 지급하는 것으로 변경되었으며, 참전유공자 유족수당은 원안대로 참전유공자의 배우자에게 매월 5만원 지급입니다. 조례 개정안, 신ㆍ구조문대비표, 관계법령 발췌는 붙임을 참고하시기 바랍니다. 성별영향분석평가 결과 특이사항은 없으며, 규제심사 해당사항 없습니다. 조례ㆍ규칙심의회는 2018년 2월 9일 수정 가결되었습니다. 비용추계서는 붙임을 참고하시고, 1월 26일부터 2월2일까지 입법예고한 결과 의견제출 없었습니다. 제안자 의견입니다. 참전유공자와 그 유족에게 합당한 예우를 하여 군민의 나라사랑정신 함양에 이바지 하고자 본 조례를 개정하는 것으로 원안대로 심의ㆍ의결하여 주시기 바랍니다. 다음은 음성군 국가보훈대상자 예우 및 지원에 관한 조례 일부개정조례안에 대하여 설명드리겠습니다. 개정이유는 국가보훈대상자 유족에 대한 합당한 예우를 하기 위하여 보훈예우수당 중 유족수당의 수급대상자를 배우자와 배우자가 없는 경우 보훈대상자의 부모까지 확대하여 생활안정에 기여하고자 하는 내용입니다. 주요 내용으로는 유족수당의 대상을 유족인 배우자에서 배우자가 없는 경우 보훈대상자의 부모까지 확대하는 것입니다. 조례 개정안, 신ㆍ구조문대비표, 관계법령 발췌는 붙임을 참고하시기 


summary:



'음성군 참전유공자 지원 조례 일부개정조례안은 참전 유공자와 그 유족에게 합당한 예우를 하기 위하여 참전명예수당을 매월 10만원으로 인상하고 참전유의공자가 사망한 경우 그 배우자에게 매월 5만원을 지급하는 내용이며, 해당 안건은 가결됨. 음성군 국가보훈대상자 예우 및 지원에 관한 조례 일부 개정조례안이 가결되었음.'

In [8]:
id=66
print('Context:\n')
display(test_real['title_context'][id])

print('\nsummary:\n')
display(sub['summary'][id])

Context:



'제235회 완주군의회(제2차 정례회) 제1차 본회의: 다음은 의사일정 제3항 완주군수 및 관계공무원 출석요구의 건을 상정합니다. 본 건에 대하여 발의하신 정종윤 의원님은 나오셔서 제안설명 해주시기 바랍니다. 존경하는 최등원 의장님과 선배 동료 의원 여러분! 그리고 박성일 군수님을 비롯한 관계 공무원 여러분! 구이·상관·소양 출신 더불어 민주당 정종윤 의원입니다. 본 안건은 지방자치법 제42조제2항 및 완주군의회 회의규칙 제66조 규정에 따른 것으로 이번 제2차 정례회 기간 중에 실시되는 군정질문에 대해서 집행부 측의 책임 있는 답변을 듣기 위해 다가오는 11월 30일 제2차 본회의에 완주군수 등 관계 공무원들의 출석을 요구하는 것입니다. 군정질문은 집행부가 추진 중인 주요 현안사업 등에 대한 질문으로 그동안 우리 의원님들께서 의정활동을 통해 수집한 자료와 군민들의 의견을 바탕으로 집행부에서 추진하거나 계획 중인 시책들의 문제점을 도출하고 발전적 대안을 제시함은 물론, 군민들의 대의기관인 완주군의회의 의사를 집행부에 전달하기 위한 것으로 집행부의 책임 있는 답변을 기대하며 완주군수 및 관계공무원 출석요구의 건에 대한 제안 설명을 마치겠습니다. 아무쪼록 제안설명 드린 원안대로 의결될 수 있도록 우리 의원님들의 협조를 부탁드립니다. 그러면 정종윤 의원님께서 제안 설명한 안건에 대하여 이의가 없으면 원안대로 의결하고자 하는데 이의가 없으므로 가결되었음을 선포합니다.'


summary:



'제235회 완주군의회 제2차 정례회 제1차 본회의에 완주군수 등 관계공무원의 출석을 요구하여 가결됨.'